Consider the standard incomplete markets model and answer the following:

Write a python program that returns the recursive competitive equilibrium for a economy with
the following parameters:

   * intertemporal discount factor($\beta$) = 0.98;
   * CRRA utility function with $\sigma$ = 2;
   * depreciation rate $\delta$= 0.08;
   * production function: $f(K,L) = K^{\alpha}L^{1-\alpha}$
   * $\alpha$ = 0.44
   * state-transition matrix: 
   * $\pi(y'|y)=$ $$\begin{bmatrix} 0.4 & 0.5 & 0.1 \\ 0.3 & 0.2 & 0.5 \\ 0.2 & 0.4 & 0.4 \end{bmatrix}$$.

Answer:

Importing modules

In [1]:
import numpy as np

The recursive problem is a little different than we were doing so far. A stationary recursive competitive equilibrium
is a value function $v:Z \times M \rightarrow \mathbb{R} $, policy function $a': Z \rightarrow \mathbb{R}$ and
consumption $c: Z \rightarrow \mathbb{R}$, policy functions for the firm $K$ and $L$, prices $r,w$ and a mesure 
$\phi \in M$ such that:
* $v,a', c$ are mesurable with respect to $B(Z)$, v satisfies the household Bellman's equation:
* $v(a,y,\phi)$ = $max_{c\geq 0, a' \geq 0}$ $(U(c) + \beta \sum_{y \in Y} \pi(y'|y)v(a', y';\phi))$
s.t $c + a' = wy + (1+r)a$ where $a',c$ are the associated policy functions for a given r and w.
* $K, L$ satisfy, given $r$ and $w$:
* $r =$ $F_{k}(K,L)$ - $\delta$
* $w =$ $F_{L}(K, L)$
* $K' \equiv a'(a,y) d\phi = K$ (because its stationary)
* $L$ = $\int y $d$\phi$
* $\int c(a,y,\phi)d\phi$ + $\int a'(a,y,\phi)d\phi$ = $F[K(\phi), L(\phi)] + (1-\delta)K(\phi)$

When something appears with a line e.g: $a'$ it stands for the future value of this parameter in this
case, tomorrow assets distribution. Every future distribution is defined by the associated policy function 
for that variable. $r$ is the interest rate, $w$ is the wage, $K$ and $L$ are the capital stock and labor supply 
respectively.

Now we will solve this problem for case asked. We already know that for the standard CRRA utility function we have:
$u(c) = \frac{c^{1-\delta}-1}{1-\delta}$. But now $\delta$ stands for depreciation rate and our risk aversion turned 
in $\sigma$ so now we have, $u(c) = \frac{c^{1-\sigma}-1}{1-\sigma}$.

In [2]:
def util_func(x):
    return (x**(1-sigma))/(1-sigma)

Note that we didn't defined $\sigma$ , we will it next. We will create a function that assigns values for the parameters
needed to solve the equilibrium.

In [3]:
def parameters_objects():
    global beta, sigma,delta,alpha,pi,y_domain, a_domain, V, iG, n_y, n_a, w,r
    beta = 0.98
    sigma = 2
    delta = 0.08
    alpha = 0.44
    pi = np.array([[0.4 , 0.5 , 0.1], 
                   [0.3 , 0.2 , 0.5 ], 
                   [0.2 , 0.4 , 0.4]])
    r = 0.05 #we will need some interest rate to maximize our objective function
    w = 1 #we will need the wage too
    # the income domain was not defined we will create one to make sure our program works properly 
    y_min = 1e-2
    y_max = 1
    n_y = len(pi) #must have one income for each state
    y_domain = np.linspace(y_min , y_max, n_y)
    # neither the assets domain was defined
    a_max = 10
    n_a = 9 #three assets distributions for each income state
    # we need a non-Ponzi condition for this case
    barA = 0 # no borrow allowed
    a_domain = np.linspace(-barA, a_max, n_a)
    
    #Now we just neeed some place to store our value function and policy function
    V = np.zeros((n_y, n_a)) #value
    iG = np.zeros((n_y, n_a), dtype = np.int) #policy  
 
    

Now we need to build our objective function: $U(c) + \beta \sum_{y \in Y} \pi(y'|y)v(a', y';\phi)$

In [4]:
def build_objective(V):
    global n_y, n_a, w, r
    F_OBJ = np.zeros((n_y, n_a, n_a)) #one dimension to each income, asset, and future asset
    #looping through all income, assets and future assets
    for i_y in range(n_y):
        y = y_domain[i_y]
        for i_a in range(n_a):
            a = a_domain[i_a]
            for i_a_line in range(n_a): #i_a_line stands for future assets
                aa = a_domain[i_a_line]
                c = w*y + a - aa/(1+r) #consumption 
                if c <= 0:
                    F_OBJ[i_y, i_a, i_a_line] = -np.inf
                else:
                    F_OBJ[i_y, i_a, i_a_line] = util_func(c) + beta*(np.dot(pi[i_y, :],V[:, i_a_line]))
    return F_OBJ

Solving the problem:

In [5]:
def maximize_TV_IG(F_OBJ):
    #maximizing for time t
    TV = np.zeros((n_y, n_a))
    T_iG = np.zeros((n_y, n_a), dtype = np.int)
    for i_y, y in enumerate(y_domain):
        for i_a, a in enumerate(a_domain):
            TV[i_y, i_a] = np.max(F_OBJ[i_y, i_a, :]) # max value of f_obj 
            T_iG[i_y, i_a] = np.argmax(F_OBJ[i_y, i_a, :]) # position associated to (y,a) pair that maximizes F_OBJ
    return TV, T_iG

Computing the stationary state: 

In [6]:
def compute_V_G_est():
    global V
    norm, tol = 2, 1e-7
    while norm>tol:
        F_OBJ = build_objective(V)
        TV, T_iG = maximize_TV_IG(F_OBJ)
        norm = np.max(abs(TV - V))
        V = np.copy(TV)
        iG = np.copy(T_iG)
    return V, iG      

Now we will introduce future assets and heterogeneity, introducing an endogenous transition matrix function:

In [7]:
def compute_Q(iG):
    #This function gives a markov transition function, Q_{r} to compute, a stationary measure phi_{r}
    #associated associated to this transition function
    Q = np.zeros((n_y*n_a, n_y*n_a))
    for i_y in range(n_y):
        for i_a in range(n_a):
            c_state = i_y*n_a + i_a
            for i_y_line in range(n_y):
                for i_a_line in range(n_a):
                    n_state = i_y_line*n_a + i_a_line
                    if iG[i_y, i_a] == i_a_line:
                        Q[c_state, n_state] += pi[i_y, i_y_line]
    return Q

Now we compute $\phi_{r}$

In [8]:
def compute_phi(Q):
    global phi
    phi = np.ones(n_y*n_a) / (n_y*n_a)
    norm_Q, tol_Q = 1, 1e-6
    while norm_Q > tol_Q:
        T_phi = np.dot(phi, Q)
        norm_Q = max(abs(T_phi - phi))
        phi = T_phi/sum(T_phi)
    return phi

We need a few more things to compute the equilibrium:

In [9]:
# Expected savings(assets)
def compute_Ea(phi):
    Ea = 0
    for i_y in range(n_y):
        for i_a in range(n_a):
            s_index = iG[i_y, i_a] 
            savings = a_domain[s_index] 
            t_index = i_y * n_a + i_a
            size = phi[t_index]
            Ea += savings*size
    return Ea

Labor supply

In [10]:
def compute_L():
    L = 0
    for i_y in range(n_y):
        for i_a in range(n_a):
            labor_supply = y_domain[i_y]
            t_index = i_y*n_a + i_a
            size_l = phi[t_index]
            L += labor_supply*size_l
    return L

The production function is $f(K,L) = K^{\alpha}L^{1-\alpha}$. As we've seen: 
\begin{equation}
r = F_{k}(K,L) - \delta
\end{equation}
So: 
\begin{equation}
k = (\alpha/(r+\delta))^{(1/(1-\alpha)}
\end{equation}.

In [11]:
def compute_k(r):
    k =(alpha/(r+delta))**(1/(1-alpha))
    return k 

Similarly:
\begin{equation}
w = (1-\alpha) \times (k^{\alpha})
\end{equation}

In [12]:
def compute_w(k):
    w = (1-alpha)*(k**alpha)
    return w

When we reach the equilibrium there should be no excess demand :
$d = K - E(a) = 0$. And finally we can compute the equilibrium.

In [13]:
def compute_d(phi):
    k = compute_k(r)
    L = compute_L()
    K = k*L
    ea = compute_Ea(phi)
    d = K - ea
    return d

In [14]:
def compute_equilibrium():
    global r, w, V, iG, Q, phi, L, k
    rho = beta**(-1)-1
    r_1, r_2 = -delta, rho #interest rate domain.
    norm_r, tol_r = 1, 1e-10
    while norm_r>tol_r:
        r = (r_1+r_2)/2
        k = compute_k(r)
        V, iG = compute_V_G_est()
        Q = compute_Q(iG)    
        phi = compute_phi(Q)    
        d = compute_d(phi)
        if d>0:
            r_1 = r
        elif d<0:
            r_2 = r
        norm_r = abs(r_1-r_2)
        #printing the interest rate at each step 
        print('[d,r_L,r_H,norm]=[{:9.6f},{:9.6f},{:9.6f},{:9.6f}]'.format(d,r_1,r_2,norm_r))

In [15]:
parameters_objects()
compute_equilibrium()

<ipython-input-3-8bad40f2643f>:26: DeprecationWarning: `np.int` is a deprecated alias for the builtin `int`. To silence this warning, use `int` by itself. Doing this will not modify any behavior and is safe. When replacing `np.int`, you may wish to use e.g. `np.int64` or `np.int32` to specify the precision. If you wish to review your current use, check the release note link for additional information.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  iG = np.zeros((n_y, n_a), dtype = np.int) #policy
<ipython-input-5-d39ec0f74c3b>:4: DeprecationWarning: `np.int` is a deprecated alias for the builtin `int`. To silence this warning, use `int` by itself. Doing this will not modify any behavior and is safe. When replacing `np.int`, you may wish to use e.g. `np.int64` or `np.int32` to specify the precision. If you wish to review your current use, check the release note link for additional information.
Deprecated in Num

[d,r_L,r_H,norm]=[25.618551,-0.029796, 0.020408, 0.050204]
[d,r_L,r_H,norm]=[12.419544,-0.004694, 0.020408, 0.025102]
[d,r_L,r_H,norm]=[ 4.430990, 0.007857, 0.020408, 0.012551]
[d,r_L,r_H,norm]=[ 3.337801, 0.014133, 0.020408, 0.006276]
[d,r_L,r_H,norm]=[ 2.863614, 0.017270, 0.020408, 0.003138]
[d,r_L,r_H,norm]=[ 2.642114, 0.018839, 0.020408, 0.001569]
[d,r_L,r_H,norm]=[ 2.534993, 0.019624, 0.020408, 0.000784]
[d,r_L,r_H,norm]=[ 2.482308, 0.020016, 0.020408, 0.000392]
[d,r_L,r_H,norm]=[ 2.456181, 0.020212, 0.020408, 0.000196]
[d,r_L,r_H,norm]=[ 2.443171, 0.020310, 0.020408, 0.000098]
[d,r_L,r_H,norm]=[ 2.436679, 0.020359, 0.020408, 0.000049]
[d,r_L,r_H,norm]=[ 2.433436, 0.020384, 0.020408, 0.000025]
[d,r_L,r_H,norm]=[ 2.431816, 0.020396, 0.020408, 0.000012]
[d,r_L,r_H,norm]=[ 2.431006, 0.020402, 0.020408, 0.000006]
[d,r_L,r_H,norm]=[ 2.430601, 0.020405, 0.020408, 0.000003]
[d,r_L,r_H,norm]=[ 2.430398, 0.020407, 0.020408, 0.000002]
[d,r_L,r_H,norm]=[ 2.430297, 0.020407, 0.020408, 0.00000

Describe how the equilibrium interest rate depends of the intertemporal discount factor,𝛽, and intertemporal substitution/risk aversion, 𝜎.

Answer:(Change $/beta$  and see what happens)

When 𝛽=0.98, the equilibrium interest rate is 2.04, which is the same as 𝜌. Now, the agent became, impatient thus the his respective beta became lower. That is, in order to the agent give up present consumption, he needs to be compensated, in the intertemporal budget constraint context that means the equilibrium interest rate needs to be greater than before. So the respective interest rate equilibrium is about 5,9 which is a big increase compared to the variation of the betas. 